<a href="https://colab.research.google.com/drive/158zwSM__zs0caehysLinxLkjY7_naqcK?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Environment Setup
UPDATE_OPENLLM = True  #@param {type:"boolean"}
WORKSPACE = 'OpenLLM'
PROJECT_NAME = 'examples/bentoml-llama2-demo'

![ ! -d $WORKSPACE ] && echo -= Initial setup OpenLLM =- && git clone https://github.com/bentoml/OpenLLM.git
%cd $WORKSPACE

if UPDATE_OPENLLM:
  !echo -= Updating openllm =-
  !git fetch origin feat/llama-colab && git checkout feat/llama-colab && git pull

%cd {PROJECT_NAME}

!echo -= Install dependencies =-
![ -f requirements.txt ] && pip install -q -r requirements.txt

In [ ]:
#@title [optional] Check the memory, and gpu info you have
import psutil
import torch

ram = psutil.virtual_memory()
ram_total = ram.total / (1024 ** 3)
print("MemTotal: %.2f GB" % ram_total)

print("=============GPU INFO=============")
if torch.cuda.is_available():
    !/opt/bin/nvidia-smi || ture
else:
    print("GPU NOT available")
    #print("RUN `openllm models` to find modles which can runable on CPU")

In [ ]:
#@title Download llama2 model
!python runner.py
#list the model you just download
!bentoml models list -o json

In [ ]:
#@title [optional] Start the llama server locally using `bentoml` command.
#bentoml serve will use the service.py under this folder to start a llama2 server
import sys
if 'google.colab' in sys.modules:
  #using colab proxy URL
  from google.colab.output import eval_js
  print("you are in colab runtime. please try it out in %s" % eval_js("google.colab.kernel.proxyPort(8001)"))

!bentoml serve service:svc -p 8001

In [ ]:
#@title Build the llama bento using bentoml (service.py bentofile.yaml)
!bentoml build -f bentofile.yaml

In [ ]:
#@title Check the bentos you just build, and push them to bentocloud if you want
! bentoml list -o json
return_code = !bentoml cloud list-context
print(return_code)
if "colab-user" not in ''.join(return_code):
  #login bentocloud
  endpoint = input("input endpoint (like https://xxx.cloud.bentoml.com):")
  token = input("input token (please follow https://docs.bentoml.com/en/latest/bentocloud/how-tos/manage-access-token.html#creating-an-api-token):")
  !bentoml cloud login --api-token {token} --endpoint {endpoint} --context colab-user

#change to your own bentos tag
!bentoml push llama-service:zhmroycxp6v4casc  --context colab-user

In [ ]:
#@title Follow the [guide](https://www.bentoml.com/blog/deploying-llama-2-7b-on-bentocloud) to deploy this llama model on bentocloud

In [ ]:
#@title Or use bentoml client to start a deployment (config specified by deployment.json)
import bentoml
import json

return_code = !bentoml cloud list-context
if "colab-user" not in ''.join(return_code):
  print("please login first!")
else:
  client = bentoml.cloud.BentoCloudClient()
  #detailed configuration in https://docs.bentoml.com/en/latest/bentocloud/reference/deployment-creation-and-update-info.html
  #runner config
  runner = bentoml.cloud.Resource.for_runner(
      resource_instance="starter-aws-g4dn-xlarge-gpu-t4-xlarge",
      #hpa_conf={"min_replicas": 1, "max_replicas": 1},
  )
  #api-server hpa config
  api_server = bentoml.cloud.Resource.for_api_server(
      resource_instance="starter-aws-t3-2xlarge-cpu-small",
  )
  hpa_conf = bentoml.cloud.Resource.for_hpa_conf(min_replicas=1, max_replicas=1)

  res = client.deployment.create(
      deployment_name="test-llama2",
      bento="llama-service:zhmroycxp6v4casc",
      context = "colab-user",
      cluster_name = "default",
      mode="deployment",
      kube_namespace='yatai',
      runners_config={"llm-llama-runner": runner},
      api_server_config=api_server,
      hpa_conf=hpa_conf,
  )
  print(json.dumps(res, indent=4))
  #!bentoml deployment create -f deployment.json --context colab-user